<a href="https://colab.research.google.com/github/fvidalf/repo-colab/blob/main/tmdb_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip uninstall -y dgl dgl-cu101 dgl-cu102 dgl-cu110 dgl-cu111 dgl-cu113 dgl-cu116 dgl-cu117 dgl-cu121 torch torchdata torchvision torchaudio torchtune

Found existing installation: dgl 2.1.0
Uninstalling dgl-2.1.0:
  Successfully uninstalled dgl-2.1.0
Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchdata 0.9.0
Uninstalling torchdata-0.9.0:
  Successfully uninstalled torchdata-0.9.0
Found existing installation: torchtune 0.6.1
Uninstalling torchtune-0.6.1:
  Successfully uninstalled torchtune-0.6.1


In [6]:
# !pip install torch==2.4.0 torchdata==0.8.0
# !pip install dgl==2.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 142.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Using cached dgl-2.1.0-cp312-cp312-manylinux1_x86_64.whl.metadata (553 bytes)
Using cached dgl-2.1.0-cp312-cp312-manylinux1_x86_64.whl (8.6 MB)


In [1]:
import torch, dgl
print("Torch:", torch.__version__)
print("Torchdata:", __import__("torchdata").__version__)
print("DGL:", dgl.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Backend:", dgl.backend.backend_name)

/usr/local/lib/python3.12/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


FileNotFoundError: Cannot find DGL C++ graphbolt library at /usr/local/lib/python3.12/dist-packages/dgl/graphbolt/libgraphbolt_pytorch_2.4.0.so

In [ ]:
device = torch.device("cpu")

In [2]:
!nvidia-smi

Sun Nov  9 21:53:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             54W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!chmod 600 /root/.config/kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c tmdb-box-office-prediction

In [ ]:
# !unzip /content/tmdb-box-office-prediction.zip

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [ ]:
print(train_df.columns)

In [ ]:
excluded_columms = ['id', 'imdb_id', 'tagline']
train_df.head(5).loc[:, ~train_df.columns.isin(excluded_columms)]

### Normalización de dataset

Primero, dividimos el dataset en varias tablas, según las entidades identificadas

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Add a split indicator
train_df["split"] = "train"
test_df["split"] = "test"

# Ensure both have the same schema
for col in set(train_df.columns) - set(test_df.columns):
    test_df[col] = None

full_df = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
import pandas as pd
import ast

# -----------------------------
# GLOBAL error log
# -----------------------------
errors = []

# -----------------------------
# Safe parsing
# -----------------------------
def safe_parse(cell, column_name, movie_id):
    if pd.isna(cell) or cell == "":
        return None
    try:
        return ast.literal_eval(cell)
    except Exception as e:
        errors.append({
            "movie_id": movie_id,
            "column": column_name,
            "error": str(e),
            "value": str(cell)[:300]
        })
        return None

# -----------------------------
# Generic normalization
# -----------------------------
def normalize_column(df, column_name, entity_id_field):
    """Normalize a column containing list/dict JSON-like data."""
    entity_rows, join_rows = [], []

    for movie_id, cell in zip(df.index, df[column_name]):
        parsed = safe_parse(cell, column_name, movie_id)
        if not parsed:
            continue
        if isinstance(parsed, dict):
            parsed = [parsed]

        for entry in parsed:
            entity_id = entry.get(entity_id_field)
            if entity_id is None:
                errors.append({
                    "movie_id": movie_id,
                    "column": column_name,
                    "error": f"Missing {entity_id_field}",
                    "value": str(entry)
                })
                continue

            entity_rows.append(entry)
            join_rows.append({"movie_id": movie_id, entity_id_field: entity_id})

    entity_df = pd.DataFrame(entity_rows).drop_duplicates(subset=[entity_id_field])
    join_df = pd.DataFrame(join_rows).drop_duplicates()
    return entity_df.reset_index(drop=True), join_df.reset_index(drop=True)

# -----------------------------
# Specialized normalization for cast/crew
# -----------------------------
def normalize_people(df, column_name, role_type):
    """Extract people (cast or crew) as separate entity + join tables."""
    person_rows, join_rows = [], []

    for movie_id, cell in zip(df.index, df[column_name]):
        parsed = safe_parse(cell, column_name, movie_id)
        if not parsed:
            continue

        for entry in parsed:
            pid = entry.get("id")
            if not pid:
                errors.append({
                    "movie_id": movie_id,
                    "column": column_name,
                    "error": "Missing person id",
                    "value": str(entry)
                })
                continue

            person_rows.append({
                "person_id": pid,
                "name": entry.get("name"),
                "gender": entry.get("gender"),
                "profile_path": entry.get("profile_path")
            })

            if role_type == "cast":
                join_rows.append({
                    "movie_id": movie_id,
                    "person_id": pid,
                    "character": entry.get("character"),
                    "order": entry.get("order"),
                    "credit_id": entry.get("credit_id"),
                    "cast_id": entry.get("cast_id")
                })
            else:
                join_rows.append({
                    "movie_id": movie_id,
                    "person_id": pid,
                    "department": entry.get("department"),
                    "job": entry.get("job"),
                    "credit_id": entry.get("credit_id")
                })

    people_df = pd.DataFrame(person_rows).drop_duplicates(subset=["person_id"])
    join_df = pd.DataFrame(join_rows).drop_duplicates()
    return people_df.reset_index(drop=True), join_df.reset_index(drop=True)



In [ ]:
from collections import namedtuple

def normalize_dataframe(df):
    """Normalize a movie metadata dataframe into relational tables."""
    df = df.copy()
    df.rename(columns={"Keywords": "keywords"}, inplace=True)

    # --- Define normalization schema ---
    Column = namedtuple(
        "Column",
        ["name", "entity_id_field", "entity_df_name", "join_df_name", "is_people"]
    )

    columns_to_normalize = [
        Column("genres", "id", "genres", "movie_genres", False),
        Column("production_companies", "id", "companies", "movie_companies", False),
        Column("keywords", "id", "keywords", "movie_keywords", False),
        Column("belongs_to_collection", "id", "collections", "movie_collections", False),
        Column("cast", "id", "people", "movie_cast", True),
        Column("crew", "id", "people", "movie_crew", True),
        Column("production_countries", "iso_3166_1", "countries", "movie_countries", False),
        Column("spoken_languages", "iso_639_1", "languages", "movie_languages", False),
    ]

    final_dataframes = {}
    all_people = []

    # --- Normalize all relational / list-like columns ---
    for col in columns_to_normalize:
        print(f"Normalizing {col.name}...")
        if col.is_people:
            people_df, join_df = normalize_people(df, col.name, col.name)
            all_people.append(people_df)
            final_dataframes[col.join_df_name] = join_df
        else:
            entity_df, join_df = normalize_column(df, col.name, col.entity_id_field)
            entity_col = f"{col.entity_df_name[:-1]}_id"
            entity_df.rename(columns={col.entity_id_field: entity_col}, inplace=True)
            join_df.rename(columns={col.entity_id_field: entity_col}, inplace=True)
            final_dataframes[col.entity_df_name] = entity_df
            final_dataframes[col.join_df_name] = join_df

        # Drop the column from the base df after it’s processed
        if col.name in df.columns:
            df.drop(columns=[col.name], inplace=True)

    # --- Merge all people into a single unique table ---
    if all_people:
        people_df = pd.concat(all_people).drop_duplicates(subset=["person_id"])
        final_dataframes["people"] = people_df

    # --- Now the remaining df is our movies table ---
    movies_df = df.copy()

    # Rename ID column
    if "id" in movies_df.columns:
        movies_df.rename(columns={"id": "movie_id"}, inplace=True)

    # -- Process table by dropping and renaming columns ---
    cols_to_ignore = ["homepage", "poster_path"]
    movies_df.drop(columns=[c for c in cols_to_ignore if c in movies_df.columns], inplace=True, errors="ignore")

    movies_df.reset_index(drop=True, inplace=True)
    if "movie_id" in movies_df.columns:
        movies_df["movie_id"] = pd.to_numeric(movies_df["movie_id"], errors="coerce")

    final_dataframes["movies"] = movies_df

    # --- Wrap up ---
    print(f"✅ Normalization complete. {len(errors)} errors logged.")
    if errors:
        log_name = "normalization_errors.csv"
        pd.DataFrame(errors).to_csv(log_name, index=False)
        print(f"⚠️ Errors saved to {log_name}")

    return final_dataframes

In [ ]:
full_dfs = normalize_dataframe(full_df)

In [ ]:
print(full_dfs.keys())

In [ ]:
def preview_final_dataframes(final_dataframes, n=10):
    """Prints the first n rows of each dataframe in final_dataframes."""
    for name, df in final_dataframes.items():
        print("=" * 80)
        print(f"📄 DataFrame: {name}  |  Shape: {df.shape}")
        print("-" * 80)
        # Show first n rows (truncate long values for clarity)
        with pd.option_context('display.max_columns', None,
                               'display.width', 1000,
                               'display.max_colwidth', 80):
            print(df.head(n))
        print("\n")

preview_final_dataframes(full_dfs)


### Construcción de grafo heterogéneo

Usando dgl, construimos el grafo declarando entidades y relaciones

In [ ]:
import dgl
import torch

def build_dgl_graph(dfs):
    # --- Create contiguous ID maps ---
    id_maps = {}
    for name, df in dfs.items():
        if name.startswith("movie_"):
            continue
        if name == "movies":
            id_maps["movie"] = {mid: i for i, mid in enumerate(df["movie_id"].values)}
        elif name == "people":
            id_maps["person"] = {pid: i for i, pid in enumerate(df["person_id"].values)}
        elif name == "genres":
            id_maps["genre"] = {gid: i for i, gid in enumerate(df["genre_id"].values)}
        elif name == "companies":
            id_maps["company"] = {cid: i for i, cid in enumerate(df["companie_id"].values)}
        elif name == "collections":
            id_maps["collection"] = {cid: i for i, cid in enumerate(df["collection_id"].values)}
        elif name == "keywords":
            id_maps["keyword"] = {kid: i for i, kid in enumerate(df["keyword_id"].values)}
        elif name == "countries":
            id_maps["country"] = {cid: i for i, cid in enumerate(df["countrie_id"].values)}
        elif name == "languages":
            id_maps["language"] = {lid: i for i, lid in enumerate(df["language_id"].values)}

    # --- Build edge data ---
    relations = {
        ('movie', 'has_genre', 'genre'): ('movie_genres', 'movie_id', 'genre_id'),
        ('movie', 'produced_by', 'company'): ('movie_companies', 'movie_id', 'companie_id'),
        ('movie', 'has_keyword', 'keyword'): ('movie_keywords', 'movie_id', 'keyword_id'),
        ('movie', 'has_cast', 'person'): ('movie_cast', 'movie_id', 'person_id'),
        ('movie', 'has_crew', 'person'): ('movie_crew', 'movie_id', 'person_id'),
        ('movie', 'belongs_to_collection', 'collection'): ('movie_collections', 'movie_id', 'collection_id'),
        ('movie', 'made_in', 'country'): ('movie_countries', 'movie_id', 'countrie_id'),
        ('movie', 'in_language', 'language'): ('movie_languages', 'movie_id', 'language_id'),
    }

    graph_data = {}

    for (src_type, etype, dst_type), (join_name, src_field, dst_field) in relations.items():
        if join_name not in dfs:
            continue
        join_df = dfs[join_name]
        src_ids = join_df[src_field].map(id_maps[src_type]).dropna().astype(int)
        dst_ids = join_df[dst_field].map(id_maps[dst_type]).dropna().astype(int)
        graph_data[(src_type, etype, dst_type)] = (src_ids.values, dst_ids.values)

    # --- Create heterogeneous graph ---
    g = dgl.heterograph(graph_data)
    return g, id_maps


In [ ]:
g, id_maps = build_dgl_graph(full_dfs)
print(g)

### Generación de features

Mediante 3 handlers: Numeric, categorical y text, codificamos features para todas las entidades, dependiendo del tipo de columnas que manejen

In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

# ------------------------------------------------------------
# Common encoders for numeric, categorical, text
# ------------------------------------------------------------
def scale_numeric(df, cols):
    df = df.copy()
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    scaler = StandardScaler()
    return scaler.fit_transform(df[cols].fillna(0.0))

def encode_categorical(df, cols):
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    return ohe.fit_transform(df[cols].fillna("UNK"))

def encode_text(df, col):
    texts = df[col].fillna("").astype(str).values
    features = np.zeros((len(texts), 2), dtype=np.float32)
    for i, t in enumerate(texts):
        features[i, 0] = len(t)
        features[i, 1] = sum(c.isupper() for c in t) / max(len(t), 1)
    scaler = StandardScaler()
    return scaler.fit_transform(features)

# ------------------------------------------------------------
# MOVIE NODE FEATURES
# ------------------------------------------------------------
def add_movie_features(g, dfs, id_maps):
    movies = dfs["movies"].copy().set_index("movie_id")

    numeric_cols = ["budget", "popularity", "runtime", "revenue"]
    cat_cols     = ["original_language", "status"]

    num_feats = scale_numeric(movies, numeric_cols)
    cat_feats = encode_categorical(movies, cat_cols)
    X = np.concatenate([num_feats, cat_feats], axis=1)

    g.nodes["movie"].data["feat"] = torch.tensor(X, dtype=torch.float32)

    # Masks + label
    train_ids = movies.query("split == 'train'").index.map(id_maps["movie"]).dropna().astype(int).values
    test_ids  = movies.query("split == 'test'").index.map(id_maps["movie"]).dropna().astype(int).values
    train_mask = torch.zeros(g.num_nodes("movie"), dtype=torch.bool)
    test_mask  = torch.zeros(g.num_nodes("movie"), dtype=torch.bool)
    train_mask[train_ids] = True
    test_mask[test_ids]   = True

    g.nodes["movie"].data["train_mask"] = train_mask
    g.nodes["movie"].data["test_mask"]  = test_mask
    labels = movies["revenue"].fillna(0.0).values
    g.nodes["movie"].data["label"] = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

    print(f"✅ movie features: {X.shape[1]} dims, {len(train_ids)} train / {len(test_ids)} test")

# ------------------------------------------------------------
# GENERIC TEXTUAL NODE FEATURES
# ------------------------------------------------------------
def add_textual_features(g, dfs, id_maps, entity_name, df_key, text_col):
    df = dfs[df_key]
    feats = encode_text(df, text_col)
    g.nodes[entity_name].data["feat"] = torch.tensor(feats, dtype=torch.float32)
    print(f"✅ {entity_name} features: {feats.shape[1]} dims ({len(df)} nodes)")

# ------------------------------------------------------------
# PEOPLE NODE FEATURES
# ------------------------------------------------------------
def add_people_features(g, dfs, id_maps):
    df = dfs["people"].copy()
    df["gender"] = pd.to_numeric(df["gender"], errors="coerce").fillna(0)
    gender_feat = df["gender"].to_numpy().reshape(-1, 1)
    name_feats = encode_text(df, "name")
    X = np.concatenate([gender_feat, name_feats], axis=1)
    g.nodes["person"].data["feat"] = torch.tensor(X, dtype=torch.float32)
    print(f"✅ person features: {X.shape[1]} dims ({len(df)} nodes)")

# ------------------------------------------------------------
# EDGE FEATURES (using same encoding functions)
# ------------------------------------------------------------
def add_edge_features(g, dfs, id_maps, edge_type, df_key, numeric_cols=None, cat_cols=None, text_cols=None):
    df = dfs[df_key].copy()
    src_t, etype, dst_t = edge_type
    src_map = id_maps[src_t]
    dst_map = id_maps[dst_t]

    df = df[df.iloc[:,0].map(src_map).notna() & df.iloc[:,1].map(dst_map).notna()]
    feature_blocks = []

    if numeric_cols:
        feature_blocks.append(scale_numeric(df, numeric_cols))
    if cat_cols:
        feature_blocks.append(encode_categorical(df, cat_cols))
    if text_cols:
        blocks = [encode_text(df, c) for c in text_cols]
        feature_blocks.extend(blocks)

    if not feature_blocks:
        feature_blocks = [np.zeros((len(df), 1))]

    feats = np.concatenate(feature_blocks, axis=1)
    g.edges[edge_type].data["feat"] = torch.tensor(feats, dtype=torch.float32)
    print(f"✅ edge {etype}: {feats.shape[1]} dims ({len(df)} edges)")

# ------------------------------------------------------------
# Attach features for all entities
# ------------------------------------------------------------
add_movie_features(g, normalized, id_maps)

add_textual_features(g, normalized, id_maps, "genre",      "genres",      "name")
add_textual_features(g, normalized, id_maps, "company",    "companies",   "name")
add_textual_features(g, normalized, id_maps, "keyword",    "keywords",    "name")
add_textual_features(g, normalized, id_maps, "collection", "collections", "name")
add_textual_features(g, normalized, id_maps, "country",    "countries",   "name")
add_textual_features(g, normalized, id_maps, "language",   "languages",   "name")
add_people_features(g, normalized, id_maps)

# ------------------------------------------------------------
# Attach edge features (same handlers)
# ------------------------------------------------------------
add_edge_features(g, normalized, id_maps, ('movie','has_cast','person'),
                  'movie_cast',
                  numeric_cols=['order'],
                  text_cols=['character'])

add_edge_features(g, normalized, id_maps, ('movie','has_crew','person'),
                  'movie_crew',
                  cat_cols=['department', 'job'])

# Dummy zeros for other relations (consistency)
for etype in g.etypes:
    if "feat" not in g.edges[etype].data:
        g.edges[etype].data["feat"] = torch.zeros((g.num_edges(etype), 1))

# ------------------------------------------------------------
# Save and summarize
# ------------------------------------------------------------
dgl.save_graphs("movie_graph_full.bin", [g])
print("\n💾 Saved → movie_graph_full.bin")

for ntype in g.ntypes:
    print(f"Node[{ntype:10}] feat_dim={g.nodes[ntype].data['feat'].shape[1]}")
for etype in g.etypes:
    print(f"Edge[{etype:25}] feat_dim={g.edges[etype].data['feat'].shape[1]}")


In [ ]:
def export_graph_to_csv(final_dataframes, output_dir="graph_data"):
    import os
    os.makedirs(output_dir, exist_ok=True)

    # Export node tables
    node_types = ["movies", "people", "genres", "companies", "collections", "keywords"]
    for node in node_types:
        if node in final_dataframes:
            final_dataframes[node].to_csv(f"{output_dir}/{node}.csv", index=False)

    # Export edge tables
    for name, df in final_dataframes.items():
        if name.startswith("movie_"):
            df.to_csv(f"{output_dir}/{name}.csv", index=False)

    print(f"✅ Graph exported to '{output_dir}/' as CSVs.")

In [ ]:
# import shutil
# shutil.rmtree("/content/graph_data")

In [ ]:
export_graph_to_csv(full_dfs)